In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

**Read Data**

In [2]:
df_train = pd.read_csv('../csvFiles/clean.csv')

In [3]:
df_train.columns

Index([u'Unnamed: 0', u'link', u'image', u'addr', u'zip', u'redfin_price',
       u'sold_price', u'Beds', u'Bath', u'SqFt', u'YearBuilt', u'SoldTime',
       u'Disc', u'Stories', u'Style', u'Community', u'County', u'LotSize',
       u'ElemSchool', u'MidSchool', u'HighSchool', u'Walk', u'Transit',
       u'Bike'],
      dtype='object')

In [4]:
print(df_train.iloc[5]['Disc'])

BOM! Buyer got cold feet. Bring your buyers to this charming well maintained home with lots of character located in the Hosford-Abernethy neighborhood with a walk score of 90! Recently refinished hardwoods in living and dining room, new roof on detached garage, updated double-paned vinyl windows, wired for security system. Basement has potential to double the living space or turn into ADU. [Home Energy Score = 5. HES Report at https://api.greenbuildingregistry.com/report/pdf/R183302-20180524.pdf]


In [6]:
import sys
#sys.path.insert(0, '/home/ooo/projects/housingprice/code')
sys.path.insert(0, '/home/gangwu/cs224n/housingprice/code')
import vocab
from vocab import get_glove

In [7]:
glove_path = '../data/glove/glove.6B.50d.txt'
embedding_size = 50

In [8]:
emb_matrix, word2id, id2word = get_glove(glove_path, embedding_size)

  0%|          | 1758/2196017 [00:00<02:04, 17573.48it/s]

Loading GLoVE vectors from file: ../data/glove/glove.6B.50d.txt


 18%|█▊        | 400000/2196017 [00:10<00:47, 37502.85it/s]


In [9]:
print(len(emb_matrix))
print(len(word2id))
print(len(id2word))

2196019
400002
400002


In [10]:
print(emb_matrix[0])
print(next(iter(word2id)))
print(word2id['sky'])

[-0.47851833  0.5579875  -0.0570473   0.18844255 -0.26922183 -0.01966445
 -0.38515593 -0.23353382 -1.4116583  -0.94986905  1.39031972  0.01340189
 -1.08604726 -1.13297584  0.48419735  0.58259373  0.31675727 -0.73535243
  0.02780522  1.52671851 -0.45642692 -0.43885422  0.85493782 -0.59238879
 -0.58410456 -2.07340075 -0.34491365 -0.34557877 -0.24347448 -1.02403083
  1.41462456 -0.20856926  1.05463165  1.6520875   0.51867926 -0.04733679
 -0.9136005  -1.42055527 -0.06208297 -0.11008329 -1.20923822 -0.86434786
 -1.18914725 -1.45249156  0.22917996  0.79803887 -0.61456111 -0.34830932
 -0.47532453  1.85809705]
biennials
3507


In [11]:
sentence = df_train.iloc[5]['Disc']
print(sentence)

BOM! Buyer got cold feet. Bring your buyers to this charming well maintained home with lots of character located in the Hosford-Abernethy neighborhood with a walk score of 90! Recently refinished hardwoods in living and dining room, new roof on detached garage, updated double-paned vinyl windows, wired for security system. Basement has potential to double the living space or turn into ADU. [Home Energy Score = 5. HES Report at https://api.greenbuildingregistry.com/report/pdf/R183302-20180524.pdf]


In [12]:
from vocab import PAD_ID, UNK_ID
import re

def split_by_whitespace(sentence):                               
    words = []                                                             
    for space_separated_fragment in sentence.strip().split():       
        words.extend(re.split(" ", space_separated_fragment))              
    return [w for w in words if w] 

def cleanup_tokens(tokens):
    words = []
    for tk in tokens:
        tmp = ''
        for c in tk:
            if not c.isalnum():
                continue
            tmp += c.lower()
        if tmp:
            words.append(tmp)
    return words

def sentence_to_token_ids(sentence, word2id):                              
    """Turns an already-tokenized sentence string into word indices        
    e.g. "i do n't know" -> [9, 32, 16, 96]
    Note any token that isn't in the word2id mapping gets mapped to the id for UNK
    """
    tokens = split_by_whitespace(sentence) # list of strings
    clean_tokens = cleanup_tokens(tokens)
    ids = [word2id.get(w, UNK_ID) for w in clean_tokens]                         
    return tokens, clean_tokens, ids

In [13]:
tokens, clean_tokens, ids = sentence_to_token_ids(sentence, word2id)

In [14]:
print(tokens)
print(clean_tokens)
print(ids)

['BOM!', 'Buyer', 'got', 'cold', 'feet.', 'Bring', 'your', 'buyers', 'to', 'this', 'charming', 'well', 'maintained', 'home', 'with', 'lots', 'of', 'character', 'located', 'in', 'the', 'Hosford-Abernethy', 'neighborhood', 'with', 'a', 'walk', 'score', 'of', '90!', 'Recently', 'refinished', 'hardwoods', 'in', 'living', 'and', 'dining', 'room,', 'new', 'roof', 'on', 'detached', 'garage,', 'updated', 'double-paned', 'vinyl', 'windows,', 'wired', 'for', 'security', 'system.', 'Basement', 'has', 'potential', 'to', 'double', 'the', 'living', 'space', 'or', 'turn', 'into', 'ADU.', '[Home', 'Energy', 'Score', '=', '5.', 'HES', 'Report', 'at', 'https://api.greenbuildingregistry.com/report/pdf/R183302-20180524.pdf]']
['bom', 'buyer', 'got', 'cold', 'feet', 'bring', 'your', 'buyers', 'to', 'this', 'charming', 'well', 'maintained', 'home', 'with', 'lots', 'of', 'character', 'located', 'in', 'the', 'hosfordabernethy', 'neighborhood', 'with', 'a', 'walk', 'score', 'of', '90', 'recently', 'refinished'

In [15]:
def padded(token_batch, batch_pad=0):
    """
    Inputs:
      token_batch: List (length batch size) of lists of ints.
      batch_pad: Int. Length to pad to. If 0, pad to maximum length sequence in token_batch.
    Returns:
      List (length batch_size) of padded of lists of ints.
        All are same length - batch_pad if batch_pad!=0, otherwise the maximum length in token_batch
    """
    maxlen = max(map(lambda x: len(x), token_batch)) if batch_pad == 0 else batch_pad
    return map(lambda token_list: token_list + [PAD_ID] * (maxlen - len(token_list)), token_batch)

In [16]:
padded_ids = padded([ids], 10)

In [17]:
print(padded_ids)

[[57343, 7877, 407, 1868, 1239, 940, 394, 4489, 6, 39, 12389, 145, 3130, 165, 19, 4233, 5, 1397, 707, 8, 2, 1, 2403, 19, 9, 2387, 1656, 5, 1888, 746, 115574, 61288, 8, 758, 7, 7212, 929, 52, 4060, 15, 15974, 7613, 6060, 1, 11194, 2213, 12661, 12, 196, 280, 7672, 33, 1158, 6, 1280, 2, 758, 865, 48, 892, 77, 47044, 165, 636, 1656, 424, 69409, 257, 24, 1]]


In [19]:
source activate house

SyntaxError: invalid syntax (<ipython-input-19-7d1797729e38>, line 1)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

ImportError: No module named torch